# Data Preparation

In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
# setting display format so that large values are shown properly
pd.set_option('display.float_format', lambda x: '%.4f' % x)

sns.set_style(style='dark')
sns.set_context("notebook")

In [159]:
housing = pd.read_csv('train.csv')

In [160]:
housing.shape

(1460, 81)

In [161]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [162]:
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [163]:
housing.index = housing['Id']

housing = housing.drop('Id',axis=1)

In [164]:
def get_columns_with_nan_percentage(df):
    nan_cols = [{
            "column":c,
            "percentage":round(100*(df[c].isnull().sum()/len(df[c].index)), 2),
            "type":df[c].dtype
            } for c in df.columns if round(100*(df[c].isnull().sum()/len(df[c].index)), 2)>0]
    return pd.DataFrame.from_records(nan_cols)

In [165]:
get_columns_with_nan_percentage(housing)

,column,percentage,type
0,LotFrontage,17.7400,float64
1,Alley,93.7700,object
2,MasVnrType,0.5500,object
3,MasVnrArea,0.5500,float64
4,BsmtQual,2.5300,object
5,BsmtCond,2.5300,object
6,BsmtExposure,2.6000,object
7,BsmtFinType1,2.5300,object
8,BsmtFinType2,2.6000,object
9,Electrical,0.0700,object


We can drop columns with more 80% null values

In [166]:
housing = housing.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1)

In [167]:
get_columns_with_nan_percentage(housing)

,column,percentage,type
0,LotFrontage,17.7400,float64
1,MasVnrType,0.5500,object
2,MasVnrArea,0.5500,float64
3,BsmtQual,2.5300,object
4,BsmtCond,2.5300,object
5,BsmtExposure,2.6000,object
6,BsmtFinType1,2.5300,object
7,BsmtFinType2,2.6000,object
8,Electrical,0.0700,object
9,FireplaceQu,47.2600,object


Deleting rows where `MasVnrArea` and `GarageYrBlt` is NAN

In [168]:
housing = housing[~housing['MasVnrArea'].isnull()]
housing = housing[~housing['GarageYrBlt'].isnull()]

Let us analyze `LotFrontage`

In [169]:
housing['LotFrontage'].describe()

count   1121.0000
mean      70.6655
std       24.2668
min       21.0000
25%       60.0000
50%       70.0000
75%       80.0000
max      313.0000
Name: LotFrontage, dtype: float64

Here, we can see that the `mean` is almost equivalent to 75%, so we can replace the missing values to `mean`.

In [170]:
housing['LotFrontage'] = housing['LotFrontage'].fillna(70)

In [171]:
get_columns_with_nan_percentage(housing)

,column,percentage,type
0,BsmtQual,2.1900,object
1,BsmtCond,2.1900,object
2,BsmtExposure,2.2600,object
3,BsmtFinType1,2.1900,object
4,BsmtFinType2,2.2600,object
5,Electrical,0.0700,object
6,FireplaceQu,44.7800,object


In [172]:
housing = housing.fillna('NA')

In [173]:
get_columns_with_nan_percentage(housing)

""


Now, we can see that there is no NAN in the dataset

There are some columns which are numeric but according to the data dictionary they are categorical. We can convert these columns as categorical.

In [174]:
housing.select_dtypes(include=['float64','int64']).columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

Following columns can be converted to categorical.

1. MSSubClass
2. OverallQual
3. OverallCond
4. BsmtFullBath
5. BsmtHalfBath
6. FullBath
7. HalfBath
8. BedroomAbvGr
9. KitchenAbvGr
10. TotRmsAbvGrd
11. Fireplaces
12. GarageCars
13. MoSold
14. YrSold
15. GarageYrBlt
16. YearBuilt
17. YearRemodAdd

In [175]:
def convert_to_category(columns):
    for column in columns:
        housing[column] = housing[column].astype('category')

In [176]:
convert_to_category([
    'MSSubClass', 'OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath',
    'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
    'Fireplaces', 'GarageCars', 'MoSold', 'YrSold', 'GarageYrBlt', 'YearBuilt',
    'YearRemodAdd'
])

In [177]:
def get_int_float_columns_with_Zero_percentage(df):
    nan_cols = [{
            "column":c,
            "percentage":round(100*((df[c]==0).sum()/len(df[c].index)), 2),
            "type":df[c].dtype
            } for c in df.columns if round(100*((df[c]==0).sum().sum()/len(df[c].index)), 2)>0]
    return pd.DataFrame.from_records(nan_cols)

In [178]:
get_int_float_columns_with_Zero_percentage(housing.select_dtypes(include=['float64','int64']))

,column,percentage,type
0,MasVnrArea,57.3300,float64
1,BsmtFinSF1,30.9300,int64
2,BsmtFinSF2,88.1100,int64
3,BsmtUnfSF,7.5100,int64
4,TotalBsmtSF,2.1900,int64
5,2ndFlrSF,56.3100,int64
6,LowQualFinSF,98.6100,int64
7,WoodDeckSF,50.6900,int64
8,OpenPorchSF,43.6900,int64
9,EnclosedPorch,86.5100,int64


We can delete the columns which have more 50% **0** values as shown above.

In [179]:
housing = housing.drop([
    'MasVnrArea', 'BsmtFinSF2', '2ndFlrSF', 'LowQualFinSF', 'WoodDeckSF',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal'
],
                       axis=1)

## Analysis of the data

Reference for the following methods: https://towardsdatascience.com/a-starter-pack-to-exploratory-data-analysis-with-python-pandas-seaborn-and-scikit-learn-a77889485baf#89dd and the previous assignments.

In [180]:
default_figsize = (10, 5)

In [181]:
def categorical_summarized(dataframe,
                           x=None,
                           y=None,
                           hue=None,
                           palette='Set1',
                           verbose=True,
                           figsize=default_figsize,
                           title="",
                           xlabel=None,
                           ylabel=None,
                           rotate_labels=False):
    '''
    Helper function that gives a quick summary of a given column of categorical data
    Arguments
    =========
    dataframe: pandas dataframe
    x: str. horizontal axis to plot the labels of categorical data, y would be the count
    y: str. vertical axis to plot the labels of categorical data, x would be the count
    hue: str. if you want to compare it another variable (usually the target variable)
    palette: array-like. Colour of the plot
    Returns
    =======
    Quick Stats of the data and also the count plot
    '''
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = dataframe[column_interested]

    if verbose:
        print(series.describe())
        print('mode: ', series.mode())
        print('=' * 80)
        print(series.value_counts())

    sns.set(rc={'figure.figsize': figsize})
    sorted_df = dataframe.sort_values(column_interested)
    ax = sns.countplot(x=x, y=y, hue=hue, data=sorted_df)

    plt.title(title)
    if not xlabel:
        xlabel = column_interested
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    total = len(dataframe[column_interested])
    if rotate_labels:
        plt.setp(ax.get_xticklabels(),
                 rotation=30,
                 horizontalalignment='right')
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height() / 2
        ax.annotate(percentage, (x, y))
    plt.tight_layout()
    plt.style.use('fivethirtyeight')
    plt.show()

In [182]:
def quantitative_summarized(dataframe,
                            x=None,
                            y=None,
                            hue=None,
                            palette='Set1',
                            ax=None,
                            verbose=True,
                            swarm=False,
                            figsize=default_figsize):
    '''
    Helper function that gives a quick summary of quantattive data
    Arguments
    =========
    dataframe: pandas dataframe
    x: str. horizontal axis to plot the labels of categorical data (usually the target variable)
    y: str. vertical axis to plot the quantitative data
    hue: str. if you want to compare it another categorical variable (usually the target variable if x is another variable)
    palette: array-like. Colour of the plot
    swarm: if swarm is set to True, a swarm plot would be overlayed
    Returns
    =======
    Quick Stats of the data and also the box plot of the distribution
    '''
    series = dataframe[y]
    print(series.describe())
    if verbose:
        print('mode: ', series.mode())
        print('=' * 80)
        print(series.value_counts())
    sns.set(rc={'figure.figsize': figsize})

    sns.boxplot(x=x, y=y, hue=hue, data=dataframe, palette=palette, ax=ax)

    if swarm:
        sns.swarmplot(x=x,
                      y=y,
                      hue=hue,
                      data=dataframe,
                      palette=palette,
                      ax=ax)
    plt.tight_layout()
    plt.style.use('fivethirtyeight')
    plt.show()

In [183]:
def plot_column(df,
                col,
                chart_type='Hist',
                dtype=int,
                bins=25,
                figsize=default_figsize):
    temp_df = df[col]
    sns.set(rc={'figure.figsize': figsize})
    if chart_type == 'Hist':
        ax = sns.countplot(temp_df)
    elif chart_type == 'Dens':
        ax = sns.distplot(temp_df)
    xmin, xmax = ax.get_xlim()
    ax.set_xticks(np.round(np.linspace(xmin, xmax, bins), 2))
    plt.tight_layout()
    plt.locator_params(axis='y', nbins=6)
    plt.style.use('fivethirtyeight')
    plt.show()

In [184]:
def univariate_analysis(col,
                        chart_type='Dens',
                        df=housing,
                        is_categorical=False,
                        title="",
                        xlabel=None,
                        ylabel=None,
                        rotate_labels=False,
                        bins=25):
    if is_categorical:
        categorical_summarized(df,
                               x=col,
                               title=title,
                               xlabel=xlabel,
                               ylabel=ylabel,
                               rotate_labels=rotate_labels,
                               verbose=False)
    else:
        quantitative_summarized(df, y=col, verbose=False)
        plot_column(df, col, chart_type=chart_type, bins=bins)

In [185]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1371 entries, 1 to 1460
Data columns (total 66 columns):
MSSubClass       1371 non-null category
MSZoning         1371 non-null object
LotFrontage      1371 non-null float64
LotArea          1371 non-null int64
Street           1371 non-null object
LotShape         1371 non-null object
LandContour      1371 non-null object
Utilities        1371 non-null object
LotConfig        1371 non-null object
LandSlope        1371 non-null object
Neighborhood     1371 non-null object
Condition1       1371 non-null object
Condition2       1371 non-null object
BldgType         1371 non-null object
HouseStyle       1371 non-null object
OverallQual      1371 non-null category
OverallCond      1371 non-null category
YearBuilt        1371 non-null category
YearRemodAdd     1371 non-null category
RoofStyle        1371 non-null object
RoofMatl         1371 non-null object
Exterior1st      1371 non-null object
Exterior2nd      1371 non-null object
MasVnrTyp

In [186]:
housing['SalePrice'].describe()

count     1371.0000
mean    185181.8906
std      78872.3527
min      35311.0000
25%     133950.0000
50%     167000.0000
75%     217750.0000
max     755000.0000
Name: SalePrice, dtype: float64